In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [2]:
# trainEND = '2024-12-31'
# testSTART = '2025-01-01'

In [3]:
# import datetime

# # принимает строку, приращивает дату, возвращает строку
# # по умолчанию возвращает следующий день
# def NextDay(date_str, delta=1):
#     date_time = datetime.datetime.strptime(date_str, "%Y-%m-%d").date() + datetime.timedelta(days=delta)
#     return date_time.strftime("%Y-%m-%d")

In [ ]:
import os
import pandas as pd
import numpy as np

# Путь к папке с данными
data_dir = "../data"

# Чтение данных из файла data_compare_eda.csv в папке data
file_path = os.path.join(data_dir, "data_compare_eda.csv")
df = pd.read_csv(file_path, parse_dates=True, index_col=0)

# Список тикеров
tickers = ["BNBUSDT", "BTCUSDT", "CAKEUSDT", "ETHUSDT",
           "LTCUSDT", "SOLUSDT", "STRKUSDT", "TONUSDT",
           "USDCUSDT", "XRPUSDT", "PEPEUSDT",
           "HBARUSDT", "APTUSDT", "LDOUSDT", "JUPUSDT"]

# Переименовываем колонки в соответствии со списком тикеров
df.columns = tickers

# Заполняем пропуски последними известными значениями (forward fill)
df_h = df.fillna(method='ffill').resample('h').ffill()

# Вычисляем относительные изменения (процентное изменение)
df_h_pct = df_h.pct_change().iloc[1:]

# Вычисляем логарифмическую доходность
df_h_log = np.log(df_h / df_h.shift(1)).iloc[1:]

# Выводим первые строки логарифмической доходности
print(df_h_log.head())

In [6]:
selected_portfolio = ['JUPUSDT', 'PEPEUSDT', 'APTUSDT', 'CAKEUSDT', 'HBARUSDT', 'STRKUSDT', 'USDCUSDT'] # 'XRPUSDT'

In [7]:
df_h_log = df_h_log[selected_portfolio].dropna().copy()

In [ ]:
df_h_log

In [9]:
train_size = int(len(df_h_log) * 0.8)  # 80% данных для обучения
train_df = df_h_log.iloc[:train_size]
test_df = df_h_log.iloc[train_size:]

trainEND = df_h_log.index[train_size - 1]  # Последняя дата в обучающей выборке
testSTART = df_h_log.index[train_size]

# train_df = df_h_log[:trainEND]
# test_df = df_h_log[testSTART:]

In [10]:
# Шаг 1: Обработка пропущенных значений
train_df = train_df.interpolate(method='linear', axis=0)  # Линейная интерполяция
train_df = train_df.fillna(0)  # Заполнение оставшихся пропусков нулями

# Шаг 2: Проверка, что все активы имеют достаточное количество данных
min_obs = len(train_df) * 0.8  # Минимум 80% данных
train_df = train_df.loc[:, train_df.count() >= min_obs]

# Шаг 3: Вычисление метрик портфеля
assetsNum = len(train_df.columns)
iterNum = 10000

all_portf = np.zeros((iterNum, assetsNum))
ret_arr = np.zeros(iterNum)
vol_arr = np.zeros(iterNum)
sharpe_arr = np.zeros(iterNum)

for x in range(iterNum):
    portf = np.random.dirichlet(np.ones(assetsNum), size=1)[0]  # Генерация случайного портфеля
    
    all_portf[x, :] = portf
    
    ret_arr[x] = np.sum(train_df.mean() * portf)  # Ожидаемая доходность портфеля
    
    vol_arr[x] = np.sqrt(np.dot(portf.T, np.dot(train_df.cov(), portf)))  # Волатильность портфеля
    
    sharpe_arr[x] = ret_arr[x] / vol_arr[x]  # Коэффициент Шарпа

In [ ]:
max_sharpe_portf = sharpe_arr.argmax()
max_sharpe_ret = ret_arr[sharpe_arr.argmax()]
max_sharpe_vol = vol_arr[sharpe_arr.argmax()]

print(sharpe_arr.max())
print(max_sharpe_portf)
print(all_portf[max_sharpe_portf, :])

In [ ]:
plt.figure(figsize=(12,8))
plt.scatter(vol_arr, ret_arr, c=sharpe_arr, cmap='viridis')
plt.colorbar(label='Sharpe Ratio')
plt.xlabel('Volatility')
plt.ylabel('Return')
plt.scatter(max_sharpe_vol, max_sharpe_ret, c='red')
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
plt.ylabel('Return')
np.matmul(df_h_pct[selected_portfolio][:trainEND], all_portf[max_sharpe_portf, :]).cumsum().plot(color='green', linewidth=2)
plt.grid()
plt.show()

In [ ]:
# бэктест
plt.figure(figsize=(12,8))
plt.ylabel('Return')
np.matmul(df_h_pct[selected_portfolio][testSTART:], all_portf[max_sharpe_portf, :]).cumsum().plot(color='maroon', linewidth=2)
plt.grid()
plt.show()

In [15]:
# доходность
period_ret = 24

In [16]:
# возвращает прибыль, волатильность и коэффициент Шарпа для любого портфеля
def get_params(portf):
    portf = np.array(portf)
    ret = np.sum(df_h_log.mean() * portf) * period_ret
    vol = np.sqrt(np.dot(portf.T, np.dot(df_h_log.cov()*period_ret, portf)))
    shrp = ret/vol
    return np.array([ret, vol, shrp])

# вовзращает отрицательный кэф. Шарпа, чтобы решать задачу минимизации
def neg_sharpe(portf):
    return get_params(portf)[2] * -1

# проверка на валидность: возвращает 0, если сумма равна 1
def check_valid(portf):
    return np.sum(portf) - 1

In [17]:
cons = ({'type': 'eq', 'fun': check_valid})
bounds = tuple([(0, 1) for _ in range(assetsNum)])
init_portf = [1/assetsNum for _ in range(assetsNum)]

In [ ]:
# поиск оптимального портфеля с помощью МНК
from scipy.optimize import minimize
res = minimize(neg_sharpe, init_portf, method='SLSQP', bounds=bounds, constraints=cons)
print(res)

In [ ]:
get_params(res.x)

## Визуализация границ

In [20]:
def minimize_vol(portf):
    return get_params(portf)[1]

In [21]:
frontier_x = []
frontier_y = np.linspace(0, 0.6, 200)

for y in frontier_y:
    cons = ({'type':'eq', 'fun': check_valid},
            {'type':'eq', 'fun': lambda w: get_params(w)[0] - y})
    
    result = minimize(minimize_vol, init_portf, method='SLSQP', bounds=bounds, constraints=cons)
    frontier_x.append(result['fun'])

In [ ]:
plt.figure(figsize=(12,8))
plt.scatter(vol_arr, ret_arr, c=sharpe_arr, cmap='viridis')
plt.colorbar(label='Sharpe Ratio')
plt.xlabel('Volatility')
plt.ylabel('Return')
plt.plot(frontier_x, frontier_y, 'r--', linewidth=3)
plt.show()

## Динамика по портфелю + тестирование

In [23]:
train_din = np.matmul(df_h_pct[selected_portfolio][:trainEND], res.x).cumsum() 
test_din = np.matmul(df_h_pct[selected_portfolio][testSTART:], res.x).cumsum() 

In [ ]:
plt.figure(figsize=(12,8))
train_din.plot(linewidth=2)
plt.title('Суммарная прибыль')
plt.ylabel('Return')
plt.grid()
plt.axhline(y=0, color='black')
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
test_din.plot(linewidth=2)
plt.title('Суммарная прибыль - test')
plt.ylabel('Return')
plt.grid()
plt.axhline(y=0, color='black')
plt.show()

In [26]:
np.matmul(df_h_pct[selected_portfolio][testSTART:], res.x).to_csv('./markowitz_return_series.csv')

In [ ]:
# Доходность портфеля за период
print('Доходность портфеля за период {:.2%}'.format(test_din[-1]))
# Шарп
print('Коэффициент Шарпа {:.2f}'.format(test_din[-1] / test_din.std()))

## Ребалансировка

In [ ]:
# Сколько дней бэктеста всего
backPeriodAll = (df_h_log[selected_portfolio].index[-1] - df_h_log[selected_portfolio][testSTART:].index[0]).days

# Раз в сколько дней делать ребалансировку (раз в день)
reFreq = 7  #  ребалансировка

# Формируем все даты ребалансировки
reDates = pd.date_range(start=testSTART, end=df_h_log[selected_portfolio].index[-1], freq=f'{reFreq}D')

print(reDates)

In [ ]:
import numpy as np
import pandas as pd

# Параметры
period_ret = 24 * 30 * 3
iterNum = 5000    # Количество случайных портфелей
assetsNum = len(df[selected_portfolio].columns)  # Количество активов
reWindow = 30 * 12      # Окно обучения для ребалансировки (в днях) 560
rePortfs = {}     # Словарь для хранения оптимальных портфелей

# Генерация случайных портфелей для каждой даты ребалансировки
for d in range(len(reDates)):
    # Определяем окно обучения для текущей даты ребалансировки
    start_date = reDates[d] - pd.Timedelta(days=reWindow)  # Начало окна
    end_date = reDates[d]  # Конец окна
    retrain_df = df_h_log[selected_portfolio].loc[start_date:end_date]  # Данные за окно

    all_portf = np.zeros((iterNum, assetsNum))  # Все портфели
    ret_arr = np.zeros(iterNum)  # Доходность
    vol_arr = np.zeros(iterNum)  # Волатильность
    sharpe_arr = np.zeros(iterNum)  # Коэффициент Шарпа

    for x in range(iterNum):
        # Генерация случайных весов портфеля
        portf = np.random.random(assetsNum)
        portf = portf / np.sum(portf)  # Нормализация весов до суммы = 1
        
        all_portf[x, :] = portf  # Сохраняем веса
        
        # Расчет доходности
        ret_arr[x] = np.sum((retrain_df[selected_portfolio].mean() * portf) * period_ret)
        
        # Расчет волатильности
        vol_arr[x] = np.sqrt(np.dot(portf.T, np.dot(retrain_df[selected_portfolio].cov() * period_ret, portf)))
        
        # Расчет коэффициента Шарпа
        sharpe_arr[x] = ret_arr[x] / vol_arr[x]
    
    # Находим портфель с максимальным коэффициентом Шарпа
    max_sharpe_idx = sharpe_arr.argmax()
    max_sharpe_portf = all_portf[max_sharpe_idx, :]
    max_sharpe_ret = ret_arr[max_sharpe_idx]
    max_sharpe_vol = vol_arr[max_sharpe_idx]

    # Сохраняем оптимальный портфель для текущей даты ребалансировки
    rePortfs[reDates[d]] = list(max_sharpe_portf)

    # Выводим информацию о текущей итерации
    print(f"Дата ребалансировки: {reDates[d]}")
    print(f"Максимальный коэффициент Шарпа: {sharpe_arr.max():.4f}")
    print(f"Оптимальные веса: {max_sharpe_portf}")
    print("=" * 30)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Визуализация на процентной доходности
retest_din = None  # Инициализируем переменную для хранения результатов

for d in range(len(reDates)):
    # Определяем временной интервал для текущего периода тестирования
    start_date = reDates[d]
    if d < len(reDates) - 1:
        end_date = reDates[d + 1]
    else:
        end_date = None  # Если это последний период, берем данные до конца
    
    # Берем данные за текущий период
    retest_df = df_h_pct[selected_portfolio].loc[start_date:end_date]
    
    # Вычисляем доходность портфеля с учетом оптимальных весов
    portfolio_return = np.matmul(retest_df, rePortfs[reDates[d]])
    
    # Накопленная доходность (cumsum)
    cumulative_return = pd.Series(portfolio_return).cumsum()
    
    # Объединяем данные
    if retest_din is None:
        retest_din = cumulative_return
    else:
        retest_din = pd.concat([retest_din, cumulative_return])

# Визуализация
plt.figure(figsize=(12, 8))
retest_din.plot(color='green', linewidth=2)
plt.title('Cumulative Portfolio Return')
plt.ylabel('Return')
plt.xlabel('Date')
plt.grid()
plt.axhline(y=0, color='black')  # Горизонтальная линия на уровне 0
plt.show()

In [ ]:
# Доходность портфеля за период
print('Доходность портфеля за период {:.2%}'.format(retest_din[-1]))
# Шарп
print('Коэффициент Шарпа {:.2f}'.format(retest_din[-1] / retest_din.std()))

In [ ]:
pd.DataFrame(rePortfs).T.plot(kind='bar', figsize=(12,8))
plt.legend(selected_portfolio)
plt.grid(axis='y')
plt.ylabel('Portfolio shares')
plt.xlabel('Rebalancing points')
plt.show()